<a href="https://colab.research.google.com/github/ankitgoel1602/data-science/blob/master/data-analysis/pandas-groups/pandas_apply_user_defined_functions_on_groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required libraries

In [2]:
!git clone https://github.com/ankitgoel1602/data-science.git

Cloning into 'data-science'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 178 (delta 80), reused 92 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (178/178), 9.68 MiB | 10.53 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [3]:
# import pandas, you should have Pandas version > 1.10.
import pandas as pd

# set max columns to display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

# Dataset Details

The dataset which we are going to use here is the time-series data provided by <a href="https://data.worldbank.org/"> World Bank Open data </a> and is related to the crowd-sourced price data collected from 15 countries. For more details about the data, refer <a href="https://datacatalog.worldbank.org/dataset/crowdsourced-price-data-collection-pilot"> Crowdsourced Price Data Collection Pilot </a>.For this exercise, we are going to use data collected for Argentina.

In [4]:
# importing data
initial_data = pd.read_csv('/content/data-science/datasets/arg-crowdsourcedpdcpilot02_final_obs_all_clean.csv', parse_dates=['created_at'])

In [5]:
# top 5 rows
initial_data.head(5)

,item_name,item_code,bh_name,bh_code,created_at,brand,created_fr_lat,created_fr_lon,obs_uid,size,quantity,total_size,converted_total_size,normalized_total_size,price,normalized_price,fx_usd,fx_local,normalized_price_usd,normalized_price_local,currency,units,reference_unit_of_measurement,reference_quantity,place_uuid,place_name,place_lat,place_long,store_type,country,l0,l1,l2,l0_geo,l1_geo,l2_geo,l3_geo,pop_density,dist_l2,city_radius,metadata_json
0,"Men's haircut, barber shop",111211101,Hairdressing salons and personal grooming establishments,1112111.0,2016-06-06 14:57:57.493,NaN,-34.612083,-58.424259,0003134e-cb5d-4a81-bd25-770d67c4759b,1.0,1,1.0,1.0,1.0,50.00,50.000,13.938950,13.938950,3.587071,50.000,ARS,piece(s),Service,1.0,0a9723ea-dcdd-478d-8928-4b019ac6b2d5,peluqueria gaacon 124,-34.610687,-58.424356,private_service_provider,AR,Argentina,Buenos Aires,Buenos Aires,Argentina,Ciudad de Buenos Aires,Distrito Federal,Buenos Aires,323.583880,2.397569,50,"{ ""price_includes_charges_value"" : ""yes"" }"
1,"Sardines, tinned, with skin, in vegetable oil, WKB",110113201,Preserved or processed fish and seafood,1101132.0,2016-04-19 17:47:57.557,GV,-31.466946,-64.163376,0006d339-1ef4-4d27-9f99-aac6773c4633,160.0,1,160.0,160.0,0.8,29.30,36.625,14.163950,14.163950,2.585790,36.625,ARS,g,Gram,200.0,7db0b4f4-eb09-4f0c-bbb5-c5d052064f49,walmart hoggins,-31.464056,-64.163222,small_medium_shop,AR,Argentina,Córdoba,Cordoba,Argentina,Córdoba,Capital,Córdoba,40.096850,6.264624,20,"{ ""tin_can_packaging_value"" : ""yes"", ""sardines_whole_value"" : ""headless"", ""sardines_oil_verify_value"" : ""vegetable_oil"", ""brand_value"" : ""GV"" }"
2,"Broken rice, 25%, BNR",110111106,Rice,1101111.0,2016-03-08 18:43:48.048,jumbo,-34.590607,-58.408207,0008d632-8fee-4957-972a-ceb1307e06b9,1.0,1,1.0,1.0,1.0,12.49,12.490,15.446950,15.446950,0.808574,12.490,ARS,kg,Kilogram,1.0,61a47c51-c47c-4f2a-8f6b-914ff938218c,DISCO Av. Santa Fe 3047 Palermo,-34.590394,-58.408558,NaN,AR,Argentina,Buenos Aires,Buenos Aires,Argentina,Ciudad de Buenos Aires,Distrito Federal,Buenos Aires,321.490285,4.491375,50,"{ ""packaged_unpackaged_value"" : ""prepacked"", ""rice_type_value"" : ""[cheap]"", ""rice_type_other_value"" : """", ""rice_packaging_value"" : ""paper_or_plastic_bag"", ""brand_value"" : ""jumbo"" }"
3,"Men's shirt, WKB-M",110312103,Garments,1103121.0,2016-04-10 17:33:59.277,george,-31.395527,-64.259949,0008ec67-df92-4d7a-a859-b9d0bb95cacb,1.0,1,1.0,1.0,1.0,349.00,349.000,14.461316,14.461316,24.133350,349.000,ARS,piece(s),Piece,1.0,6a9f0ff13a333d85609d738923db6c49364c0e38,Walmart Córdoba Oeste Colón,-31.394419,-64.260132,small_medium_shop,AR,Argentina,Córdoba,Cordoba,Argentina,Córdoba,Capital,Córdoba,36.855019,6.533218,20,"{ ""brand_value"" : ""george"", ""adult_child_value"" : ""adult_man"", ""features_collar_shirt_value"" : ""[buttoned_down, collar, long_sleeves]"", ""size_clothing_value"" : ""m"" }"
4,"Vacuum cleaner, WKB-M",110531111,Major household appliances whether electric or not,1105311.0,2016-03-24 21:36:48.824,philips,-34.674744,-58.367367,0008f63c-11f6-4f3b-9277-77a11288f361,1.0,1,1.0,1.0,1.0,1799.00,1799.000,14.502950,14.502950,124.043729,1799.000,ARS,piece(s),Piece,1.0,a09e2842d54603845a7d84859854729151e485cc,Walmart Avellaneda,-34.676401,-58.365980,small_medium_shop,AR,Argentina,Buenos Aires,Buenos Aires,Argentina,Buenos Aires,Avellaneda,Crucecita,118.042055,10.591399,50,"{ ""vaccuum_features_value"" : ""[automatic_cord_rewind, dust_bag, bag_full_indicator]"", ""brand_value"" : ""philips"", ""vaccuum_wattage_normalized_value"" : ""1400"", ""model_value"" : ""phi 1400 compact go"",..."


In [6]:
# Lets select few columns for this exercise
columns_to_select = ['item_name','bh_name', 'brand', 'size', 'quantity', 'price', 'store_type', 'city_radius']

In [7]:
data = initial_data[columns_to_select]

#### Data cleaning ####
# lets drop the data where brand is not there
data = data[(~data['brand'].str.contains('none', case=False, na=False)) & (~data['brand'].isna())]

# brand names have trailing spaces lets remove those
data['brand'] = data['brand'].str.strip()

In [8]:
# sample rows
data.head(5)

,item_name,bh_name,brand,size,quantity,price,store_type,city_radius
1,"Sardines, tinned, with skin, in vegetable oil, WKB",Preserved or processed fish and seafood,GV,160.0,1,29.30,small_medium_shop,20
2,"Broken rice, 25%, BNR",Rice,jumbo,1.0,1,12.49,NaN,50
3,"Men's shirt, WKB-M",Garments,george,1.0,1,349.00,small_medium_shop,20
4,"Vacuum cleaner, WKB-M",Major household appliances whether electric or not,philips,1.0,1,1799.00,small_medium_shop,50
5,"Instant coffee, NESCAFE Classic","Coffee, tea, and cocoa",Nescafé,170.0,1,112.00,small_medium_shop,20


# Getting categorical distribution per group.

To understand it better, we will go through few examples on how you can group the data and apply user defined functions to get distributions.

## Each brand, distribution and count of different store types

In [9]:
# user-defined function to get distribution
def get_distribution(column):
  # fill missing values in case there are any
  column = column.fillna("None")
  return column.value_counts(dropna=False).to_dict()

In [10]:
# pass the user defined function for the columns as part of aggregation
data.groupby('brand').agg(
    store_types_distribution = ('store_type',get_distribution),
    total_stores = ('item_name','count')
    ).reset_index().sort_values('total_stores', ascending=False).head()

,brand,store_types_distribution,total_stores
2328,tex,"{'small_medium_shop': 638, 'None': 89, 'specialized_shop': 25}",752
877,carrefour,"{'small_medium_shop': 255, 'None': 167, 'specialized_shop': 94, 'street_outlet': 37}",553
996,curitas,"{'small_medium_shop': 326, 'None': 80, 'specialized_shop': 41, 'market': 7}",454
1960,philips,"{'small_medium_shop': 115, 'None': 83, 'specialized_shop': 37, 'market': 4}",239
1059,dolphin,"{'small_medium_shop': 207, 'None': 18, 'specialized_shop': 1}",226


## Each brand, percentage of different stores they have

In [10]:
# user-defined function to get distribution percentage
def get_distribution_perc(column):
  # fill missing values in case there are any
  column = column.fillna("None")
  return ((column.value_counts(dropna=False,
                               normalize=True)*100).round(2)).to_dict()

In [11]:
# pass the user defined function for the columns as part of aggregation
data.groupby('brand').agg(store_types_distribution_perc = ('store_type',get_distribution_perc),
                          total_stores = ('item_name','count')).reset_index().sort_values('total_stores', ascending=False).head()

,brand,store_types_distribution_perc,total_stores
2328,tex,"{'small_medium_shop': 84.84, 'None': 11.84, 'specialized_shop': 3.32}",752
877,carrefour,"{'small_medium_shop': 46.11, 'None': 30.2, 'specialized_shop': 17.0, 'street_outlet': 6.69}",553
996,curitas,"{'small_medium_shop': 71.81, 'None': 17.62, 'specialized_shop': 9.03, 'market': 1.54}",454
1960,philips,"{'small_medium_shop': 48.12, 'None': 34.73, 'specialized_shop': 15.48, 'market': 1.67}",239
1059,dolphin,"{'small_medium_shop': 91.59, 'None': 7.96, 'specialized_shop': 0.44}",226


## Apply different functions to columns

In [12]:
# you can apply different functions to multiple columns or
# multiple functions to the same columns
data.groupby('brand').agg(
      store_type_distribution = ('store_type',get_distribution),
      store_type_distribution_perc = ('store_type',get_distribution_perc),
      total_stores = ('item_name','count')
    ).reset_index().sort_values('total_stores', ascending=False).head()

,brand,store_type_distribution,store_type_distribution_perc,total_stores
2328,tex,"{'small_medium_shop': 638, 'None': 89, 'specialized_shop': 25}","{'small_medium_shop': 84.84, 'None': 11.84, 'specialized_shop': 3.32}",752
877,carrefour,"{'small_medium_shop': 255, 'None': 167, 'specialized_shop': 94, 'street_outlet': 37}","{'small_medium_shop': 46.11, 'None': 30.2, 'specialized_shop': 17.0, 'street_outlet': 6.69}",553
996,curitas,"{'small_medium_shop': 326, 'None': 80, 'specialized_shop': 41, 'market': 7}","{'small_medium_shop': 71.81, 'None': 17.62, 'specialized_shop': 9.03, 'market': 1.54}",454
1960,philips,"{'small_medium_shop': 115, 'None': 83, 'specialized_shop': 37, 'market': 4}","{'small_medium_shop': 48.12, 'None': 34.73, 'specialized_shop': 15.48, 'market': 1.67}",239
1059,dolphin,"{'small_medium_shop': 207, 'None': 18, 'specialized_shop': 1}","{'small_medium_shop': 91.59, 'None': 7.96, 'specialized_shop': 0.44}",226


In [13]:
# now lets say, you would like to see for each brand and
# store what different category of items they have
brand_store_category = data.groupby(['brand','store_type']).agg(
    category_distribution = ('bh_name',get_distribution),
    category_distribution_perc = ('bh_name',get_distribution_perc),
    total_items = ('bh_name','count')
    ).reset_index().sort_values('total_items', ascending=False)
brand_store_category.head()

,brand,store_type,category_distribution,category_distribution_perc,total_items
1994,tex,small_medium_shop,"{'Garments': 636, 'Shoes and other footwear': 2}","{'Garments': 99.69, 'Shoes and other footwear': 0.31}",638
784,curitas,small_medium_shop,{'Other Medical Products': 326},{'Other Medical Products': 100.0},326
679,carrefour,small_medium_shop,"{'Garments': 31, 'Food products n.e.c.': 27, 'Rice': 24, 'Gardens and pets': 24, 'Appliances, articles and products for personal care': 21, 'Other bakery products': 18, 'Other personal effects': 1...","{'Garments': 12.16, 'Food products n.e.c.': 10.59, 'Rice': 9.41, 'Gardens and pets': 9.41, 'Appliances, articles and products for personal care': 8.24, 'Other bakery products': 7.06, 'Other person...",255
841,dolphin,small_medium_shop,{'Bicycles': 207},{'Bicycles': 100.0},207
1896,sinteplast,specialized_shop,{'Maintenance and repair of the dwelling': 139},{'Maintenance and repair of the dwelling': 100.0},139


In [14]:
# data about specific brands
brand_store_category[brand_store_category['brand']=='carrefour'].head()

,brand,store_type,category_distribution,category_distribution_perc,total_items
679,carrefour,small_medium_shop,"{'Garments': 31, 'Food products n.e.c.': 27, 'Rice': 24, 'Gardens and pets': 24, 'Appliances, articles and products for personal care': 21, 'Other bakery products': 18, 'Other personal effects': 1...","{'Garments': 12.16, 'Food products n.e.c.': 10.59, 'Rice': 9.41, 'Gardens and pets': 9.41, 'Appliances, articles and products for personal care': 8.24, 'Other bakery products': 7.06, 'Other person...",255
680,carrefour,specialized_shop,"{'Maintenance and repair of the dwelling': 46, 'Maintenance and repair of personal transport equipment': 44, 'Appliances, articles and products for personal care': 2, 'Glassware, Tableware and Hou...","{'Maintenance and repair of the dwelling': 48.94, 'Maintenance and repair of personal transport equipment': 46.81, 'Appliances, articles and products for personal care': 2.13, 'Glassware, Tablewar...",94
681,carrefour,street_outlet,"{'Rice': 17, 'Cheese': 6, 'Bread': 4, 'Mineral waters, soft drinks, fruit and vegetable juices': 3, 'Preserved or processed fish and seafood': 2, 'Jams, marmalades and honey': 2, 'Pasta products':...","{'Rice': 45.95, 'Cheese': 16.22, 'Bread': 10.81, 'Mineral waters, soft drinks, fruit and vegetable juices': 8.11, 'Preserved or processed fish and seafood': 5.41, 'Jams, marmalades and honey': 5.4...",37


# Aggregating based on the condition on the column

## Find top brands that sells garments and how many stores they have for garments.

In [15]:
# API to filter the column only on "Garments"
def count_stores_with_garments_items(column):
  column = column[column.str.contains("Garments")]
  return column.shape[0]

In [16]:
# top brands that sells garments
data.groupby(['brand']).agg(
    count_garments_stores = ('bh_name',count_stores_with_garments_items),
    total_stores = ("bh_name", 'count')
    ).sort_values('count_garments_stores', ascending=False).reset_index().head()

,brand,count_garments_stores,total_stores
0,tex,750,752
1,vincenzo,101,101
2,george,80,120
3,Tex,46,46
4,polo club,45,46


In [17]:
# find count and distribution of different men items that each brand sells
def count_store_men_items(column):
  column = column[column.str.contains("Men")]
  return column.shape[0]

def distribution_store_men_items(column):
  column = column[column.str.contains("Men")]
  return column.value_counts(dropna=False).to_dict()

In [18]:
data.groupby(['brand']).agg(
    count_store_men_items = ('item_name',count_store_men_items),
    dist_store_men_items = ('item_name', distribution_store_men_items)
    ).sort_values('count_store_men_items', ascending=False).reset_index().head()

,brand,count_store_men_items,dist_store_men_items
0,tex,752,"{'Men's blue jeans, WKB-L': 199, 'Men's trousers, cotton/polyester, WKB-L': 190, 'Men's T-shirt, WKB-L': 188, 'Men's shirt, WKB-M': 173, 'Men's lace-up shoes, WKB-L': 2}"
1,george,120,"{'Men's shirt, WKB-M': 42, 'Men's lace-up shoes, WKB-L': 40, 'Men's trousers, cotton/polyester, WKB-L': 26, 'Men's blue jeans, WKB-L': 7, 'Men's T-shirt, WKB-L': 5}"
2,vincenzo,101,"{'Men's blue jeans, WKB-L': 55, 'Men's shirt, WKB-M': 46}"
3,batistella,92,"{'Men's lace-up shoes, WKB-L': 92}"
4,tulipan,70,"{'Men's condoms, WKB': 70}"


# Ignoring values in aggregation

This is exactly opposite of what we did in the last section. We would ignore certain values and aggregate the data

## Find top brands that don't sell garments and number of stores they have matching this criteria.

In [19]:
def count_stores_without_garments_items(column):
  column = column[~column.str.contains("Garments")]
  return column.shape[0]

In [20]:
# top brands that sells non-garments items.
data.groupby(['brand']).agg(
    count_without_garments_stores = ('bh_name',count_stores_without_garments_items),
    total_stores = ("bh_name", 'count')).sort_values('count_without_garments_stores', ascending=False).reset_index().head()

,brand,count_without_garments_stores,total_stores
0,carrefour,522,553
1,curitas,454,454
2,philips,239,239
3,dolphin,226,226
4,sancor,224,224


In [11]:
# find count and distribution of different brands that sell items other than designed for Men
def count_stores_without_men_items(column):
  column = column[~column.str.contains("Men")]
  return column.shape[0]

def distribution_stores_without_men_items(column):
  column = column[~column.str.contains("Men")]
  return column.value_counts(dropna=False).to_dict()

data.groupby(['brand']).agg(
    count_stores_without_men_items =
     ('item_name',count_stores_without_men_items),
    dist_stores_without_men_items =
     ('item_name', distribution_stores_without_men_items),
    total_stores = ("item_name","count")
    ).sort_values('count_stores_without_men_items', ascending=False
                  ).reset_index().head()

,brand,count_stores_without_men_items,dist_stores_without_men_items,total_stores
0,carrefour,522,"{'Paint, indoor use, washable, WKB': 46, 'Summer tire, R15, WKB': 44, 'Cat food, tin, WKB': 40, 'Cheese, processed, WKB': 36, 'Cooking salt, WKB': 35, 'Trolley, soft case, BL': 33, 'Long-grain ric...",553
1,curitas,454,"{'Plaster strips, plastic foil, WKB': 454}",454
2,philips,239,"{'Energy saving light bulb, 15W, WKB': 182, 'Steam iron, WKB-M': 30, 'Cigarettes, domestic, blond, WKB': 14, 'Vacuum cleaner, WKB-M': 13}",239
3,dolphin,226,"{'Bicycle, basic commuter, WKB': 165, 'Hybrid bicycle, WKB': 61}",226
4,sancor,224,"{'Butter, unsalted, WKB': 86, 'Yoghurt, plain, WKB': 74, 'Cheese, processed, WKB': 57, 'Milk, fresh, unskimmed, WKB': 7}",224
